# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import os
import random

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')), sep=';', header=0)
### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,<h1>Forbidden</h1><pre>Request ID: 2022-12-22-16-47-18-592870F462E3DD91</pre>


In [3]:
os.getcwd ()

'C:\\Users\\150ho\\Desktop\\DS\\lesson4'

In [4]:
# Загружаем файл
df = pd.read_csv('Химический анализ родника в Нескучном саду.csv', sep=';')
df = df.set_index('Показатель') # столбец "Показатель" сделайте индексным
df

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [5]:
# смотрим инфу по датасету
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, pH to Окисляемость перманганатная
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Единица измерений  14 non-null     object
 1   Результат анализа  14 non-null     object
 2   Норматив           14 non-null     object
dtypes: object(3)
memory usage: 448.0+ bytes


In [6]:
# приведем в соответствие столбец "Результат анализа" в число
df["Результат анализа"] = df["Результат анализа"].str.extract(r'(\d+(?:\.\d+)?)').astype(float)
# приведем в соответствие числа столбеца "Норматив"
df["Норматив"] = df["Норматив"].str.replace(',', '.')

In [7]:
#  считаем количестово пропущенных значений
df.isna().sum()

Единица измерений    0
Результат анализа    1
Норматив             0
dtype: int64

In [8]:
# заменяем NaN на ноль по по некоректным данным
df["Результат анализа"].fillna(0, inplace = True)

In [9]:
# создадим функцию для того, чтобы программно "прочитать и понять" значение столбца "Норматив"
def Vivod(normativ, resul):
    if 'в пределах' in normativ:
        num = re.findall(r'(\d+(?:\.\d+)?)', normativ)
        if float(num[0]) <= resul <= float(num[1]):
            return 'Yes'
        else: return 'No'
    elif 'не более' in normativ:
        num = re.findall(r'(\d+(?:\.\d+)?)', normativ)
        if resul <= float(num[-1]):
            return 'Yes'
        else: return 'No'

In [10]:
# создадим итоговый столбец
df['Вывод'] = df.apply(lambda x: Vivod(x['Норматив'], x["Результат анализа"]), axis=1)

In [11]:
# вывод итог
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,Yes
Запах,баллы,1.000,не более 2-3,Yes
Цветность,градусы,0.000,не более 30,Yes
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Yes
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,Yes
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Yes
Нитраты (по NO3),мг/дм3,24.000,не более 45,Yes
Фосфаты (P),мг/дм3,0.360,не более 3.5,Yes
Хлориды (Cl),мг/дм3,200.000,не более 350,Yes


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [12]:
# заполняем ящик
Y1 = ['Orange']*5 + ['Apple']*4
Y2 = ['Orange']*4 + ['Apple']*4
Y3 = ['Orange']*3 + ['Apple']*4
n = 1000000 # количество выбора

In [13]:
# формируем выборку
df = pd.DataFrame({'A': pd.Series(np.random.choice(Y1, n)),
                   'B': pd.Series(np.random.choice(Y2, n)),
                   'C': pd.Series(np.random.choice(Y3, n))})

In [14]:
%%time
# вероятность
df[(df.A == 'Orange') & (df.B == 'Orange') & (df.C == 'Orange')].shape[0] / df.shape[0]

CPU times: total: 453 ms
Wall time: 445 ms


0.119145

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение

In [15]:
# детали
D1 = ['Стардант']*3 + ['Не Стардант']*7
D2 = ['Стардант']*3 + ['Не Стардант']*6
n = 1000000 # количество выбора

In [16]:
# формируем выборку
df = pd.DataFrame({'A': pd.Series(np.random.choice(D1, n)),
                   'B': pd.Series(np.random.choice(D2, n))})

In [17]:
%%time
# вероятность
df[(df.A == 'Не Стардант') & (df.B == 'Стардант')].shape[0] / df.shape[0]

CPU times: total: 312 ms
Wall time: 313 ms


0.233456